# Quantum Optimization Run

This notebook executes a single run of the quantum optimization pipeline. The parameters for the run are specified in a YAML file.

In [1]:
from ortools.sat.python import cp_model

In [1]:
# Minimal CP-SAT sanity check (single-thread)
from ortools.sat.python import cp_model

model = cp_model.CpModel()
x = model.NewIntVar(0, 10, "x")
y = model.NewIntVar(0, 10, "y")
model.Add(x + y == 7)
model.Maximize(3 * x + y)

solver = cp_model.CpSolver()
solver.parameters.num_search_workers = 1
solver.parameters.max_time_in_seconds = 5
status = solver.Solve(model)
print("status:", solver.StatusName(status))
if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
    print("solution:", solver.Value(x), solver.Value(y))

: 

In [2]:
import json
import logging
import os
import sys

import yaml

from pipeline.main import single_run

logger = logging.getLogger("pipeline_logger")


def configure_logger():

    # Remove all existing handlers
    if logger.hasHandlers():
        logger.handlers.clear()

    logger.setLevel(logging.INFO)

    ch = logging.StreamHandler(sys.stdout)
    ch.setLevel(logging.INFO)

    formatter = logging.Formatter('[%(asctime)s] %(levelname)s: %(message)s')
    ch.setFormatter(formatter)
    
    logger.addHandler(ch)

    # Prevent duplicate handlers if the cell is run multiple times
    # if not logger.handlers:
    #     logger.setLevel(logging.INFO)
        
    #     ch = logging.StreamHandler(sys.stdout)
    #     ch.setLevel(logging.INFO)
        
    #     formatter = logging.Formatter('[%(asctime)s] %(levelname)s: %(message)s')
    #     ch.setFormatter(formatter)
        
    #     logger.addHandler(ch)


def read_parameters(parameter_path: str) -> dict:

    if not os.path.isfile(parameter_path):
        logger.error(f"Error: file '{parameter_path}' does not exist.")
        sys.exit(1)

    try:

        with open(parameter_path, 'r') as file:
            return yaml.safe_load(file)

    except Exception as e:

        logger.error(f"Error reading parameters file: {e}")
        sys.exit(1)

In [2]:
configure_logger()

# In a notebook, we can't use sys.argv. 
# Instead, specify the path to the parameter file directly.
# You can change this to any valid parameter file.
parameter_path = 'params.yaml'

logger.info(f"reading parameters file: '{parameter_path}'")
parameter_dict = read_parameters(parameter_path)

if parameter_dict:
    output_folder = parameter_dict.get('output_folder', '.')
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    output_dict = single_run(parameter_dict)

    output_file_path = os.path.join(output_folder, "output.json")
    logger.info(f"writing output file: {output_file_path}")
    with open(output_file_path, "w") as f:
        json.dump(output_dict, f, indent=4)
else:
    logger.error("Failed to read parameter file. Halting execution.")

[2025-11-11 10:29:49,398] INFO: reading parameters file: 'params.yaml'
[2025-11-11 10:29:49,412] INFO: Using seed 12345
[2025-11-11 10:29:49,412] INFO: Using seed 12345
[2025-11-11 10:29:49,415] INFO: Output will be written in ./delme
[2025-11-11 10:29:49,415] INFO: Output will be written in ./delme
[2025-11-11 10:29:49,425] INFO: Building problem SimpleProductionPlanningProblem
[2025-11-11 10:29:49,425] INFO: Building problem SimpleProductionPlanningProblem


: 

In [4]:
configure_logger()

# In a notebook, we can't use sys.argv. 
# Instead, specify the path to the parameter file directly.
# You can change this to any valid parameter file.
parameter_path = 'params.yaml'

logger.info(f"reading parameters file: '{parameter_path}'")
parameter_dict = read_parameters(parameter_path)

if parameter_dict:
    output_folder = parameter_dict.get('output_folder', '.')
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

[2025-11-11 12:13:24,272] INFO: reading parameters file: 'params.yaml'


In [3]:
seed = parameter_dict['seed']
output_folder = parameter_dict['output_folder']
backend_name = parameter_dict['backend_name']
is_backend_fake = parameter_dict['is_backend_fake']
problem_class = parameter_dict['problem_class']
circuit_class = parameter_dict['circuit_class']
num_layers = parameter_dict['num_layers']
num_starting_points = parameter_dict['num_starting_points']
lower_bound = parameter_dict['lower_bound']
upper_bound = parameter_dict['upper_bound']
optimization_params = parameter_dict['optimization_params']
use_cache = parameter_dict['use_cache']
cache_filename = parameter_dict['cache_filename']
cache_save_every = parameter_dict['cache_save_every']
num_estimator_shots = parameter_dict['num_estimator_shots']
num_sampler_shots = parameter_dict['num_sampler_shots']
problem_params = parameter_dict['problem_params']

In [4]:
import numpy as np
from qiskit_algorithms.utils import algorithm_globals
from pipeline.plotter import Plotter

logger.info(f"Using seed {seed}")
np.random.seed(seed)
algorithm_globals.random_seed = seed

logger.info(f"Output will be written in {output_folder}")
plotter = Plotter(f"{output_folder}/plots")

[2025-11-11 11:49:52,520] INFO: Using seed 12345
[2025-11-11 11:49:52,520] INFO: Output will be written in ./delme
[2025-11-11 11:49:52,520] INFO: Output will be written in ./delme


In [5]:
from pipeline.backends import get_aer_from_backend, get_real_backend

backend = get_aer_from_backend(seed)
if backend_name:
    logger.info(f"Building backend {backend_name}")
    backend = get_real_backend(backend_name)
    if is_backend_fake:
        backend = get_aer_from_backend(seed, backend)

In [6]:
from pipeline.utils import find_most_promising_feasible_bitstring, class_importer

problem_class = "MaxCutProblem"
problem_params = {
    'n_nodes': 5,
    'density': 0.7
}

ProblemClass = class_importer("pipeline.problems", problem_class)
CircuitClass = class_importer("pipeline.qaoa_circuits", circuit_class)

logger.info(f"Building problem {problem_class}")

[2025-11-11 11:49:53,276] INFO: Building problem MaxCutProblem


In [7]:
from pipeline.problems.abstract_problem import AbstractProblem

In [8]:
problem: AbstractProblem = ProblemClass(seed, problem_params)

: 

In [ ]:

num_qubits = problem.hamiltonian.num_qubits
logger.info(f"The problem has {num_qubits} logic qubits")